In [ ]:
# import libraries
import os
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Import Data

file_path = '/content/drive/My Drive/qozmgkuw6vq8k7pq.csv'
BoardEx= pd.read_csv(file_path, encoding= 'latin', engine= 'python')

# **Data Pre-Processing**

In [ ]:
# Filtering the dataframe for only UK records

BoardEx = BoardEx[BoardEx['HOCountryName'].fillna('').str.startswith('United Kingdom')]


,RowType,DirectorName,CompanyName,BrdPosition,RoleName,FulltextDescription,NED,DirectorID,CompanyID,DateStartRole,DateEndRole,HOCountryName,Sector,OrgType,ISIN
943819,Unlisted Organisations,Jo Regan-Iles,Alpha Thrust Management Ltd,No,Chief People Officer,NaN,No,2612232,3816402,2023-07-01,C,United Kingdom - England,NaN,Private,NaN
943820,Unlisted Organisations,Andrew Mullinger,Amullinger Ltd,Yes,NED,NaN,Yes,1452267,3816418,2016-08-04,C,United Kingdom - England,NaN,Private,NaN
943821,Unlisted Organisations,Andrew Mullinger,Hollonds Woodland Management Ltd,Yes,NED,NaN,Yes,1452267,3816420,2022-04-07,C,United Kingdom - England,NaN,Private,NaN
943822,Unlisted Organisations,Martin Whitmarsh,Aston Martin Performance Technologies Ltd,Yes,Group CEO,NaN,No,1673239,3816424,2022-01-01,C,United Kingdom - England,NaN,Private,NaN
943823,Unlisted Organisations,Andrew Mullinger,Gnome Energy Ltd,Yes,NED,NaN,Yes,1452267,3816430,2023-04-03,C,United Kingdom - England,NaN,Private,NaN


In [ ]:
#Drop uncessary columns
BoardEx= BoardEx.drop(columns=['DirectorName','CompanyName','BrdPosition','RoleName','FulltextDescription','NED','HOCountryName','OrgType'])

In [ ]:
#Extract start year and end year of Director Employment

BoardEx['DateEndRole'] = BoardEx['DateEndRole'].replace(regex='C', value=2024)
BoardEx = BoardEx[BoardEx.DateStartRole != 'N']
BoardEx = BoardEx[BoardEx.DateEndRole != 'N']

BoardEx['DateStartRole']=BoardEx['DateStartRole'].astype('str')
BoardEx['StartYear']= (BoardEx['DateStartRole'].str[:4])
BoardEx['StartYear']= BoardEx['StartYear'].astype(int)

BoardEx['DateEndRole']=BoardEx['DateEndRole'].astype('str')
BoardEx['EndYear']= BoardEx['DateEndRole'].str[:4]
BoardEx['EndYear']= BoardEx['EndYear'].astype(int)

In [ ]:
### Drop records with null value in ISIN and Sector ###
BoardEx= BoardEx.dropna(subset=['ISIN'])
BoardEx= BoardEx.dropna(subset=['Sector'])


In [ ]:
# Consolidate split records for directors with a continusous service period
BoardEx = BoardEx.sort_values(by=['DirectorID', 'CompanyID', 'ISIN', 'StartYear'])
BoardEx = BoardEx.groupby(['DirectorID', 'CompanyID', 'ISIN'], as_index=False).agg({'StartYear': 'min', 'EndYear': 'max', 'Sector': 'first'})

print(BoardEx)


       DirectorID  CompanyID          ISIN  StartYear  EndYear  \
0              31        921  GB00B07YBS95       2010     2017   
1              31      12273  GB00B68K3716       2010     2018   
2              31      13785  GB0003292009       1998     2018   
3              31      20013  GB0005630420       2011     2018   
4              31      29697  GB0000684174       2000     2024   
...           ...        ...           ...        ...      ...   
51027     2872852      20844  GB0004657408       2024     2024   
51028     2873106     110151  GB00BYQ0JC66       2024     2024   
51029     2873303    2705616  GB00BDZRYX75       2024     2024   
51030     2873623    1803884  GB00B42TN250       2024     2024   
51031     2873739     110151  GB00BYQ0JC66       2024     2024   

                           Sector  
0            Investment Companies  
1            Investment Companies  
2            Investment Companies  
3               Business Services  
4            Investment Com

In [ ]:
# dimension of the dataset
BoardEx.describe()

,DirectorID,CompanyID,StartYear,EndYear
count,5.103200e+04,5.103200e+04,51032.000000,51032.000000
mean,1.541620e+06,1.059275e+06,2016.636228,2021.860205
std,8.208231e+05,1.219908e+06,5.011650,2.485713
min,3.100000e+01,2.050000e+02,1963.000000,2017.000000
25%,8.951210e+05,1.914600e+04,2014.000000,2020.000000
50%,1.611466e+06,5.506860e+05,2017.000000,2023.000000
75%,2.222793e+06,2.127413e+06,2020.000000,2024.000000
max,2.873739e+06,3.805099e+06,2024.000000,2024.000000


In [ ]:
#Create a new record for every year a director was active for the period 2017 - 2024

B1 = BoardEx[['DirectorID','CompanyID', 'StartYear','EndYear', 'ISIN', 'Sector']].copy()
BoardExY = pd.DataFrame()

for year in range(2017, 2025):
    B1 = BoardEx[['DirectorID','CompanyID', 'StartYear','EndYear', 'ISIN', 'Sector']].copy()
    mask = (B1['StartYear'] <= year) & (B1['EndYear'] >= year)
    B1 = B1[mask]
    B1['year'] = year
    BoardExY = pd.concat([BoardExY, B1], ignore_index=True)

In [ ]:
print(BoardExY)

        DirectorID  CompanyID  StartYear  EndYear          ISIN  \
0               31        921       2010     2017  GB00B07YBS95   
1               31      12273       2010     2018  GB00B68K3716   
2               31      13785       1998     2018  GB0003292009   
3               31      20013       2011     2018  GB0005630420   
4               31      29697       2000     2024  GB0000684174   
...            ...        ...        ...      ...           ...   
214966     2872852      20844       2024     2024  GB0004657408   
214967     2873106     110151       2024     2024  GB00BYQ0JC66   
214968     2873303    2705616       2024     2024  GB00BDZRYX75   
214969     2873623    1803884       2024     2024  GB00B42TN250   
214970     2873739     110151       2024     2024  GB00BYQ0JC66   

                            Sector  year  
0             Investment Companies  2017  
1             Investment Companies  2017  
2             Investment Companies  2017  
3                Busine

In [ ]:
# Count the number of distinct active directors by year
unique_directors_by_year = {}

for year in range(2017, 2025):
    active_directors = BoardExY[(BoardExY['StartYear'] <= year) & (BoardExY['EndYear'] >= year)]

    unique_directors_count = active_directors['DirectorID'].nunique()

    unique_directors_by_year[year] = unique_directors_count

unique_directors_df = pd.DataFrame(list(unique_directors_by_year.items()), columns=['Year', 'Unique Active Directors'])

print(unique_directors_df)

   Year  Unique Active Directors
0  2017                    22587
1  2018                    22896
2  2019                    22659
3  2020                    22183
4  2021                    23016
5  2022                    22965
6  2023                    22030
7  2024                    20067


In [ ]:
# Count the number of distinct active directors by year
unique_companies_by_year = {}

for year in range(2017, 2025):
    active_companies = BoardExY[(BoardExY['StartYear'] <= year) & (BoardExY['EndYear'] >= year)]

    unique_companies_count = active_companies['CompanyID'].nunique()

    unique_companies_by_year[year] = unique_companies_count

unique_companies_df = pd.DataFrame(list(unique_companies_by_year.items()), columns=['Year', 'Unique Active Companies'])

print(unique_companies_df)


   Year  Unique Active Companies
0  2017                     1647
1  2018                     1673
2  2019                     1636
3  2020                     1605
4  2021                     1668
5  2022                     1649
6  2023                     1593
7  2024                     1503


In [1]:
#Drop 'StartYear','EndYear' as replaced by the column 'year'
BoardExY= BoardExY.drop(columns=['StartYear','EndYear'])
#Drop any duplicate records (same director, company, and year)
BoardExY= BoardExY.drop_duplicates(subset=['year', 'ISIN', 'DirectorID', 'Sector'], keep='last')

NameError: name 'BoardExY' is not defined

In [ ]:
print(BoardExY)

        DirectorID  CompanyID          ISIN                      Sector  year
0               31        921  GB00B07YBS95        Investment Companies  2017
1               31      12273  GB00B68K3716        Investment Companies  2017
2               31      13785  GB0003292009        Investment Companies  2017
3               31      20013  GB0005630420           Business Services  2017
4               31      29697  GB0000684174        Investment Companies  2017
...            ...        ...           ...                         ...   ...
214966     2872852      20844  GB0004657408           Business Services  2024
214967     2873106     110151  GB00BYQ0JC66                   Insurance  2024
214968     2873303    2705616  GB00BDZRYX75  Speciality & Other Finance  2024
214969     2873623    1803884  GB00B42TN250                      Mining  2024
214970     2873739     110151  GB00BYQ0JC66                   Insurance  2024

[214971 rows x 5 columns]


### **Build Direct Connections Sub-Network**

In [ ]:
B1= BoardExY[['DirectorID','ISIN', 'year', 'Sector']].copy()
B2= BoardExY[['DirectorID','ISIN', 'year', 'Sector']].copy()
Newlist= pd.DataFrame()
FinalSN=pd.DataFrame()

In [ ]:
#Create dyads (a record for each pair of companies connected by a shared director)
FinalSN = pd.DataFrame()

for year in range(2017, 2025):
    B1 = BoardExY[['DirectorID', 'ISIN', 'year', 'Sector']].copy()
    B2 = BoardExY[['DirectorID', 'ISIN', 'year', 'Sector']].copy()
    B1 = B1[B1['year'] == year]
    B2 = B2[B2['year'] == year]
    Newlist = pd.merge(B1, B2, how='inner', on='DirectorID')
    FinalSN = pd.concat([FinalSN, Newlist], ignore_index=True)

In [ ]:
# Drop the 'year_y' column and rename 'year_x' to 'year'
FinalSN = FinalSN.drop(columns=['year_y']).rename(columns={'year_x': 'year'})

In [ ]:
# Remove self-loops (company connected to itself)
FinalSN = FinalSN[FinalSN['ISIN_x'] != FinalSN['ISIN_y']]

In [ ]:
# Create a new column with sorted ISIN pairs to handle (ISIN_x, ISIN_y) and (ISIN_y, ISIN_x) equivalence
FinalSN['sorted_ISIN'] = FinalSN.apply(lambda row: tuple(sorted([row['ISIN_x'], row['ISIN_y']])), axis=1)

In [ ]:
# Calculate the edge weight by counting the number of distinct shared directors between any pair of companies
weights = FinalSN.groupby(['sorted_ISIN', 'year'])['DirectorID'].nunique().reset_index(name='Weight')

# Merge the calculated weights back to the original DataFrame
FinalSN = pd.merge(FinalSN, weights, on=['sorted_ISIN', 'year'])


In [ ]:
# Drop duplicates based on sorted ISIN pairs and year to ensure only a single edge exists between a pair of companies (no duplicate edges)
FinalSN.drop_duplicates(subset=['sorted_ISIN', 'year'], inplace=True)


In [ ]:
# Drop the temporary sorted_ISIN column
FinalSN.drop(columns=['sorted_ISIN'], inplace=True)

In [ ]:
print(FinalSN)

        DirectorID        ISIN_x  year                    Sector_x  \
0               31  GB00B07YBS95  2017        Investment Companies   
2               31  GB00B07YBS95  2017        Investment Companies   
4               31  GB00B07YBS95  2017        Investment Companies   
6               31  GB00B07YBS95  2017        Investment Companies   
8               31  GB00B07YBS95  2017        Investment Companies   
...            ...           ...   ...                         ...   
107998     2817587  GB0031152027  2024        Investment Companies   
108000     2845609  GB00BNKT5L33  2024                   Oil & Gas   
108002     2850202  GB0007388407  2024  Speciality & Other Finance   
108004     2869566  GB00BPQY8M80  2024              Life Assurance   
108006     2869566  GB00BPQY8M80  2024              Life Assurance   

              ISIN_y                    Sector_y  Weight  
0       GB00B68K3716        Investment Companies       1  
2       GB0003292009        Investment Co

In [ ]:
file_path = '/content/drive/My Drive/FinalSN.csv'

FinalSN.to_csv(file_path, index=False)

In [ ]:
#Count the number of distinct companies by year
unique_isin_x_by_year = FinalSN.groupby('year')['ISIN_x'].nunique().reset_index()
unique_isin_x_by_year.columns = ['Year', 'Unique_ISIN_x_Count']

unique_isin_y_by_year = FinalSN.groupby('year')['ISIN_y'].nunique().reset_index()
unique_isin_y_by_year.columns = ['Year', 'Unique_ISIN_y_Count']

unique_isins_by_year = pd.merge(unique_isin_x_by_year, unique_isin_y_by_year, on='Year')

print(unique_isins_by_year)


   Year  Unique_ISIN_x_Count  Unique_ISIN_y_Count
0  2017                 1146                 1206
1  2018                 1187                 1238
2  2019                 1164                 1197
3  2020                 1151                 1172
4  2021                 1193                 1235
5  2022                 1183                 1241
6  2023                 1129                 1182
7  2024                 1023                 1088


# **Build Indirect Connections Sub-Network**

In [ ]:
SN1= FinalSN[['ISIN_x','year','ISIN_y']].copy()
SN2= FinalSN[['ISIN_x','year','ISIN_y']].copy()
SN2.rename(columns={'ISIN_x':'ISIN_z'}, inplace=True)

In [ ]:
#Create a new record for every instance two companies are connected to the same third company
SNTemp = pd.DataFrame()
SNCO3 = pd.DataFrame()

for year1 in range(2017, 2025):
    SN1 = FinalSN[['ISIN_x', 'year', 'ISIN_y']].copy()
    SN2 = FinalSN[['ISIN_x', 'year', 'ISIN_y']].copy()
    SN2.rename(columns={'ISIN_x': 'ISIN_z'}, inplace=True)

    mask = (SN1['year'] == year1)
    mask2 = (SN2['year'] == year1)

    SN1 = SN1[mask]
    SN2 = SN2[mask2]

    SNTemp = pd.merge(SN1, SN2, how='inner', on=['ISIN_y','year'])

    SNCO3 = pd.concat([SNCO3, SNTemp], ignore_index=True)

In [ ]:
file_path = '/content/drive/My Drive/SNC03v1.csv'

SNCO3.to_csv(file_path, index=False)

In [ ]:
#Calulate the weight of indirect connections by counting the mean of the total shared directors a pair of companies share with all intermediary companies

# Merge SNCO3 with FinalSN to get weights for (ISIN_x, ISIN_y)
merged_x = pd.merge(SNCO3, FinalSN, how='left',
                    left_on=['ISIN_x', 'ISIN_y', 'year'],
                    right_on=['ISIN_x', 'ISIN_y', 'year'])

# Merge SNCO3 with FinalSN to get weights for (ISIN_z, ISIN_y)
merged_z = pd.merge(SNCO3, FinalSN, how='left',
                    left_on=['ISIN_z', 'ISIN_y', 'year'],
                    right_on=['ISIN_x', 'ISIN_y', 'year'],
                    suffixes=('_x', '_z'))

# Fill missing weights with 0
merged_x['Weight'] = merged_x['Weight'].fillna(0)
merged_z['Weight'] = merged_z['Weight'].fillna(0)

# Add weights for both parts
merged_x['Total_Weight'] = merged_x['Weight'] + merged_z['Weight']

# Append Total_Weight to SNCO3
SNCO3['Weight'] = merged_x['Total_Weight'].values

# Divide by 2 to get the mean weight (each company in the pair is connected to an intermediary - 2 direct links)
SNCO3['Weight'] = SNCO3['Weight'] / 2



In [ ]:
print(SNCO3)

              ISIN_x  year        ISIN_y        ISIN_z  Weight
0       GB00B07YBS95  2017  GB00B68K3716  GB00B07YBS95     1.0
1       GB00B07YBS95  2017  GB0003292009  GB00B07YBS95     1.0
2       GB00B07YBS95  2017  GB0003292009  GB00B68K3716     1.0
3       GB00B07YBS95  2017  GB0003292009  GB0000059971     1.0
4       GB00B07YBS95  2017  GB0003292009  GB0006039597     1.0
...              ...   ...           ...           ...     ...
210924  GB00BKM6CG53  2024  GB00BN11T727  GB00BN7DVG54     1.0
210925  GB00BN7DVG54  2024  GB00BN11T727  GB00BKM6CG53     1.0
210926  GB00BN7DVG54  2024  GB00BN11T727  GB00BN7DVG54     1.0
210927  GB00BNKT5L33  2024  GB00B03CJS30  GB00BNKT5L33     1.0
210928  GB00BPQY8M80  2024  GB0003032264  GB00BPQY8M80     1.0

[210929 rows x 5 columns]


In [ ]:
#Drop records where ISIN_x = ISIN_z  or ISIN_y (intermediary company) = ISIN_z
SNCO3_filtered = SNCO3[(SNCO3['ISIN_x'] != SNCO3['ISIN_z']) & (SNCO3['ISIN_y'] != SNCO3['ISIN_z'])]

# Sum weights by year for records with the same combination of ISIN_x and ISIN_z
SNCO3= SNCO3_filtered.groupby(['ISIN_x', 'ISIN_z', 'year'])['Weight'].sum().reset_index()

In [ ]:
SNCO3

,ISIN_x,ISIN_z,year,Weight
0,AU0000004772,AU0000025231,2020,6.0
1,AU0000004772,AU0000025231,2021,7.5
2,AU0000004772,BMG022411000,2019,4.0
3,AU0000004772,BMG022411000,2020,6.0
4,AU0000004772,BMG022411000,2021,7.5
...,...,...,...,...
120999,ZAE000081949,US0463531089,2020,1.0
121000,ZAE000081949,US0463531089,2021,1.0
121001,ZAE000081949,US0463531089,2022,1.0
121002,ZAE000081949,US0463531089,2023,1.0


In [ ]:
# Rename columns
SNCO3.rename(columns={'ISIN_z': 'ISIN_y'}, inplace=True)

# Include a sector for ISIN_y
isin_to_sector = BoardEx.set_index('ISIN')['Sector'].to_dict()

SNCO3['Sector_x'] = SNCO3['ISIN_x'].map(isin_to_sector)
SNCO3['Sector_y'] = SNCO3['ISIN_y'].map(isin_to_sector)

SNCO3 = SNCO3.sort_values(by='year', ascending=True).reset_index(drop=True)

# Display the updated dataframe
print(SNCO3)


              ISIN_x        ISIN_y  year  Weight                    Sector_x  \
0       GB00BLH3CY60  GB0004339189  2017     1.0        Investment Companies   
1       GB00BMCF8689  GB0000353929  2017     2.0        Investment Companies   
2       GB0007958233  GB0001001592  2017     3.0         Aerospace & Defence   
3       GB00BMCF8689  GB0001367019  2017     1.0        Investment Companies   
4       GB00BMCF8689  GB0001765816  2017     1.0        Investment Companies   
...              ...           ...   ...     ...                         ...   
120999  GB00BMX64W89  VGG3338A1158  2024     1.0                   Insurance   
121000  GB0006928617  GB00B8C3BL03  2024     1.0                 Real Estate   
121001  GB0006928617  GB00B28HSF71  2024     1.0                 Real Estate   
121002  GB0006928617  GB00B1KJJ408  2024     1.0                 Real Estate   
121003  ZAE000081949  US0463531089  2024     1.0  Speciality & Other Finance   

                                 Sector

In [ ]:
file_path = '/content/drive/My Drive/SNC03.csv'

SNCO3.to_csv(file_path, index=False)

In [ ]:
# Calculate summary statistics for weight of indirect connections
indirect_weight_stats = SNCO3['Weight'].describe()
print("Indirect Connections Weight Statistics:\n", indirect_weight_stats)

Indirect Connections Weight Statistics:
 count    121004.000000
mean          2.043238
std           3.727244
min           1.000000
25%           1.000000
50%           1.000000
75%           1.500000
max          65.000000
Name: Weight, dtype: float64


In [ ]:
# Multiply the 'Weight' column by 2 for direct connections so that direct connections are more strongly weighted than indirect connections
FinalSN['Weight'] = FinalSN['Weight'] * 2


In [ ]:
# Calculate summary statistics for weight of direct connections
direct_weight_stats = FinalSN['Weight'].describe()
print("Direct Connections Weight Statistics:\n", direct_weight_stats)

Direct Connections Weight Statistics:
 count    34025.000000
mean         3.174372
std          8.657346
min          2.000000
25%          2.000000
50%          2.000000
75%          2.000000
max        250.000000
Name: Weight, dtype: float64


# **Build Consolidated Network**

In [ ]:
### Combine Direct and Indirect Sub-Networks into single network
FinalSN1 = pd.concat([FinalSN, SNCO3], ignore_index=False)
print(FinalSN1)


        DirectorID        ISIN_x  year                    Sector_x  \
0             31.0  GB00B07YBS95  2017        Investment Companies   
2             31.0  GB00B07YBS95  2017        Investment Companies   
4             31.0  GB00B07YBS95  2017        Investment Companies   
6             31.0  GB00B07YBS95  2017        Investment Companies   
8             31.0  GB00B07YBS95  2017        Investment Companies   
...            ...           ...   ...                         ...   
120999         NaN  GB00BMX64W89  2024                   Insurance   
121000         NaN  GB0006928617  2024                 Real Estate   
121001         NaN  GB0006928617  2024                 Real Estate   
121002         NaN  GB0006928617  2024                 Real Estate   
121003         NaN  ZAE000081949  2024  Speciality & Other Finance   

              ISIN_y                           Sector_y  Weight  
0       GB00B68K3716               Investment Companies     2.0  
2       GB0003292009       

In [ ]:
#Two companies cannot be indirectly connected if they are directly connected - drop such duplicates
FinalSN1.drop_duplicates(subset=['ISIN_x', 'ISIN_y', 'year'], keep='first', inplace=True)
print(FinalSN1)


        DirectorID        ISIN_x  year                    Sector_x  \
0             31.0  GB00B07YBS95  2017        Investment Companies   
2             31.0  GB00B07YBS95  2017        Investment Companies   
4             31.0  GB00B07YBS95  2017        Investment Companies   
6             31.0  GB00B07YBS95  2017        Investment Companies   
8             31.0  GB00B07YBS95  2017        Investment Companies   
...            ...           ...   ...                         ...   
120998         NaN  GB00BMX64W89  2024                   Insurance   
120999         NaN  GB00BMX64W89  2024                   Insurance   
121000         NaN  GB0006928617  2024                 Real Estate   
121002         NaN  GB0006928617  2024                 Real Estate   
121003         NaN  ZAE000081949  2024  Speciality & Other Finance   

              ISIN_y                           Sector_y  Weight  
0       GB00B68K3716               Investment Companies     2.0  
2       GB0003292009       

In [ ]:
file_path = '/content/drive/My Drive/FinalSN1.csv'

FinalSN1.to_csv(file_path, index=False)